In [ ]:
# Import our dependencies
import pandas as pd
import matplotlib as plt
from sklearn.datasets import make_blobs
import sklearn as skl
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import numpy as np
import os
import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib import ticker
import matplotlib.colors as mc
import statsmodels.formula.api as sm
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
# Import the RandomForestClassifier from sklearn.ensemble
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
random_forest = RandomForestClassifier()
from sklearn.naive_bayes import MultinomialNB

In [ ]:
us_df = pd.read_csv('USvideos.csv')

In [ ]:
us_df

In [ ]:
# Remove unecessary columns
del_col = ['video_id','thumbnail_link','comments_disabled','ratings_disabled','video_error_or_removed','description']
us_df = us_df.drop(del_col, axis=1)

In [ ]:
us_cat = pd.read_csv('us_category_id.csv')
us_cat

In [ ]:
us_df['tags']

In [ ]:
us_df['tags'].unique().shape[0]

In [ ]:
all_tags = []
for tgs in us_df['tags']:
    if tgs == '[none]': continue
    all_tags.extend(tgs.split('|'))
tags_freq = pd.Series(all_tags).value_counts().to_dict()

In [ ]:
sorted(tags_freq.items(), key=lambda x: x[1], reverse=True)[:15]

In [ ]:
vector = CountVectorizer()
counts = vector.fit_transform(us_df['tags'].values)

In [ ]:
NB_Model = MultinomialNB()
targets = us_df['category_id'].values
NB_Model.fit(counts,targets)

In [ ]:
X= counts
y= targets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .1)

NBtest = MultinomialNB().fit(X_train, y_train)
nb_predictions = NBtest.predict(X_test)
acc_nb = NBtest.score(X_test, y_test)
print('The Naive Bayes Algorithm scored an accuracy of', acc_nb)